In this part, we try to standarize/transform features and then drop parts of features.

We use dummy classifier as the bottom line. 

Compare the performance of different models on different sampling strategy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from itertools import cycle

In [2]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import tensorflow as tf
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import (roc_auc_score,accuracy_score, classification_report,
confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve)
from sklearn.model_selection import train_test_split,GridSearchCV

/Users/zuochaozhao/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import pandas_profiling
import eli5

/Users/zuochaozhao/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
raw_data = pd.read_csv('~/Desktop/raw_data.csv')

In [22]:
#pandas_profiling.ProfileReport(raw_data)

## Feature prepocessing 

In [5]:
y = raw_data.iloc[:,-1].values
X = raw_data.iloc[:, :-1].values

In [6]:
feature_names = raw_data.columns.values[:-1]

#### 1.Standardize feature

In [7]:
scaler = StandardScaler()
Scale_X = scaler.fit_transform(raw_data.iloc[:, :-1])
Scaled_df = pd.DataFrame(X, columns = raw_data.columns.values[:-1])
#pandas_profiling.ProfileReport(Scaled_df)

In [8]:
Scaled_names = Scaled_df.columns.values

1.1 drop features of highly correlated and skewed

In [9]:
droped_Scale_X = Scaled_df.drop(['Domestic_Duration','International_Duration','T3_Flux','T11_Flux'], axis =1)

In [10]:
droped_Scaled_names = droped_Scale_X.columns.values

#### 2. log transformation

In [11]:
log_X = np.log(raw_data+1).iloc[:,:-1]
#pandas_profiling.ProfileReport(log_X)

In [12]:
log_names = log_X.columns.values

2.1 Transform features with high proportions of zero into 0-1 variables 

In [13]:
log_X_trans = log_X.copy()
f = lambda x: 1 if x>0 else 0
log_X_trans['Transfered_Duration'] = log_X_trans['Transfered_Duration'].apply(f)
log_X_trans['International_Duration'] = log_X_trans['International_Duration'].apply(f)
log_X_trans['T3_Flux'] = log_X_trans['T3_Flux'].apply(f)
log_X_trans['T11_Flux'] = log_X_trans['T11_Flux'].apply(f)

2.2 drop features

In [14]:
dropped_log_X_trans= log_X_trans.drop(['Domestic_Duration'],axis=1)

In [15]:
dropped_log_names = dropped_log_X_trans.columns.values

## Adjusted feature performances in machine learning models

In [17]:
# Build up Cross validation pipeline
def Pipeline_CV(X, y, model):
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0,test_size=0.3)

    # preparing oversampling data 
    os = SMOTE(random_state=1) 
    os_X_train,os_y_train=os.fit_sample(X_train,y_train)
    
    print ('withoutsampling :')
    # model for withoutsampling data 
    #Use cross validation training data
    trained_model1 = cv_train_model (model, X_train, y_train, cv_folder=4)
    #Predict on the test set
    preds1 = trained_model1.predict_proba(X_test)[:,1]
    auc1 = roc_auc_score(y_test,preds1)
    print ('Test AUC score :',auc1)
    # model for oversampling data 
    
    print ('oversampling :')
    #Use cross validation training data 
    trained_model2 = cv_train_model (model, os_X_train, os_y_train, cv_folder=4)
    #Predict on the test set
    preds2 = trained_model2.predict_proba(X_test)[:,1]
    auc2 =roc_auc_score(y_test,preds2)
        
    print ('Test AUC score :',auc2)
    print ('')

# Use cross validation training data 
def cv_train_model (model, X_train, y_train, cv_folder):
    X_train = np.array (X_train)
    y_train = np.array (y_train).ravel()
    kfolder=StratifiedKFold(y_train, n_folds= cv_folder, shuffle=True, random_state=1)
    grand_auc =0.0
    i = 0
    for train_index, test_index in kfolder:
        X_trained, X_cv = X_train[train_index], X_train[test_index]
        y_trained, y_cv = y_train[train_index], y_train[test_index]
        model.fit(X_trained, y_trained)
        preds = model.predict_proba(X_cv)[:,1]
        auc=roc_auc_score(y_cv,preds)        
        grand_auc +=auc      
        i +=1
    
    grand_auc /= cv_folder
    print ('Validation AUC score is ',grand_auc)
    return model

In [24]:
print('DummyClassifier:')
model = DummyClassifier()
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)

DummyClassifier:
Unscaled or Untransformed features
withoutsampling :
Validation AUC score is  0.48442074012703673
Test AUC score : 0.5104153712224847
oversampling :
Validation AUC score is  0.49707953603976796
Test AUC score : 0.5555279194130083

Standarized features
withoutsampling :
Validation AUC score is  0.5057061903851464
Test AUC score : 0.4897400820793434
oversampling :
Validation AUC score is  0.502068489367578
Test AUC score : 0.472142768312399

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.4867743717205192
Test AUC score : 0.506311404054222
oversampling :
Validation AUC score is  0.4961702568351284
Test AUC score : 0.415588857107325

Log transformed features
withoutsampling :
Validation AUC score is  0.49197592037898585
Test AUC score : 0.5290386767814947
oversampling :
Validation AUC score is  0.5041224799779067
Test AUC score : 0.5169133192389006

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.5213151837116212

The model performance is relatively stable and the Adjusted features do not contribute to Dummy Classifier too much.

In [27]:
print('KNeighbors:')
model = KNeighborsClassifier()
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)

KNeighbors:
Unscaled or Untransformed features
withoutsampling :
Validation AUC score is  0.5049645004263894
Test AUC score : 0.47009078472826765
oversampling :
Validation AUC score is  0.9423155335323778
Test AUC score : 0.37255938316129833

Standarized features
withoutsampling :
Validation AUC score is  0.5523892601460952
Test AUC score : 0.5089541101853003
oversampling :
Validation AUC score is  0.9589841000070092
Test AUC score : 0.5335157318741449

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.4946537293140441
Test AUC score : 0.47226713095386147
oversampling :
Validation AUC score is  0.941415612633842
Test AUC score : 0.3685486879741326

Log transformed features
withoutsampling :
Validation AUC score is  0.5412085218702866
Test AUC score : 0.4595199602039547
oversampling :
Validation AUC score is  0.9640393862446356
Test AUC score : 0.3939808481532148

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.5658377224129089
T

As KNeighbors is measured by distance; it is clear that the Standarized features performs best (0.5335) in KNeighbors algorithm. The unscaled or log transform gives score worse than that of dummy classfiers.

In [36]:
print('LogisticRegression:')
model = LogisticRegression(penalty='l1', C= 0.05,class_weight='balanced', random_state =0)
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)
eli5.show_weights(model, feature_names=dropped_log_names)

LogisticRegression:
Unscaled or Untransformed features
withoutsampling :


/Users/zuochaozhao/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Validation AUC score is  0.5792292594177433
Test AUC score : 0.5141151598059942
oversampling :
Validation AUC score is  0.7404328832828783
Test AUC score : 0.5035443352816814

Standarized features
withoutsampling :
Validation AUC score is  0.5632189593063661
Test AUC score : 0.5698296231811963
oversampling :
Validation AUC score is  0.7089144370487264
Test AUC score : 0.563860216390996

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.5628994555568706
Test AUC score : 0.5412262156448202
oversampling :
Validation AUC score is  0.7210230851500588
Test AUC score : 0.5291008581022261

Log transformed features
withoutsampling :
Validation AUC score is  0.6138573294973461
Test AUC score : 0.5268001492351697
oversampling :
Validation AUC score is  0.7484607596867547
Test AUC score : 0.5524188533764457

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.6084513491203632
Test AUC score : 0.541412759607014
oversampling :
Validation AUC scor

Weight?,Feature
+0.261,T7_Flux
+0.119,T6_Flux
+0.055,Calling_Duration
+0.049,T5_Flux
+0.034,T2_Flux
-0.034,T9_Flux
-0.038,T10_Flux
-0.107,T4_Flux
-0.160,T1_Flux
-0.264,Contact_Diversity


The Dropped Log transformed features gives best performance (0.5712) in LogisticRegression as the log transformed distrubution is not highly skewed

In [55]:
print('RandomForest:')
model = RandomForestClassifier(max_features= 0.6, max_depth =20, random_state =0,class_weight='balanced')
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)
eli5.show_weights(model, feature_names=dropped_log_names)

RandomForest:
Unscaled or Untransformed features
withoutsampling :
Validation AUC score is  0.5457107046502242
Test AUC score : 0.45003730879243875
oversampling :
Validation AUC score is  0.9933161518112035
Test AUC score : 0.41953737097375954

Standarized features
withoutsampling :
Validation AUC score is  0.545120519497073
Test AUC score : 0.45069021266011694
oversampling :
Validation AUC score is  0.9950743551055128
Test AUC score : 0.5561497326203207

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.5199401309819702
Test AUC score : 0.4815321477428181
oversampling :
Validation AUC score is  0.9935536534824284
Test AUC score : 0.45131202586742947

Log transformed features
withoutsampling :
Validation AUC score is  0.5357600125640722
Test AUC score : 0.46757244123865194
oversampling :
Validation AUC score is  0.9957923753632363
Test AUC score : 0.6478049993781868

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.54866300745650

Weight,Feature
0.1761 ± 0.1316,Visiting_Area_Diversity
0.1254 ± 0.0770,T7_Flux
0.1150 ± 0.0964,T6_Flux
0.0643 ± 0.0491,T2_Flux
0.0620 ± 0.0417,T8_Flux
0.0602 ± 0.0356,Roam_Ratio
0.0486 ± 0.0426,T5_Flux
0.0479 ± 0.0367,T1_Flux
0.0477 ± 0.0474,Contact_Area_Diversity
0.0463 ± 0.0354,T9_Flux


The Log transformed features gives best performance (0.6478) in RandomForest  as log transformed distrubution is not highly skewed

In [18]:
model = RandomForestClassifier(max_features= 0.6, max_depth =20, random_state =0,class_weight='balanced')
print('Log transformed features')
Pipeline_CV(log_X, y, model)
eli5.show_weights(model, feature_names=log_names)

Log transformed features
withoutsampling :
Validation AUC score is  0.5357600125640722
Test AUC score : 0.46757244123865194
oversampling :
Validation AUC score is  0.9957923753632363
Test AUC score : 0.6478049993781868



Weight,Feature
0.2840 ± 0.0940,Visiting_Area_Diversity
0.1133 ± 0.0670,T7_Flux
0.0758 ± 0.0692,Roam_Ratio
0.0663 ± 0.0815,T6_Flux
0.0654 ± 0.0503,T9_Flux
0.0601 ± 0.0517,T8_Flux
0.0460 ± 0.0222,Contact_Diversity
0.0418 ± 0.0324,T10_Flux
0.0400 ± 0.0416,T2_Flux
0.0395 ± 0.0289,T1_Flux
